In [21]:
# !!pip3 install pandas
import pandas as pd

In [22]:
course = pd.read_csv("../csv/Course.csv")
content = pd.read_csv("../csv/Content.csv")
content_id = pd.read_csv("../csv/content_with_doid.csv")
content_id = content_id.rename(columns={"video_code": "content_code", "doID":"content_do_id"})
module = pd.read_csv("../csv/module.csv")

In [23]:
# Fill NaN values in course_name and course_code using forward fill
content[['course_name', 'course_code']] = content[['course_name', 'course_code']].fillna(method='ffill')

/var/folders/1y/4098srp972733z2bdwdhtr447rh153/T/ipykernel_10491/3377816051.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  content[['course_name', 'course_code']] = content[['course_name', 'course_code']].fillna(method='ffill')


In [24]:
# Function to remove rows where content_name contains 'QUIZ'
def remove_quiz(df):
    return df[~df['content_name'].str.contains('QUIZ', na=False)]

# Apply the function to the content dataframe
# content = remove_quiz(content)
print("Content DataFrame after removing QUIZ rows:")
content


Content DataFrame after removing QUIZ rows:


,course_name,course_code,content_name,content_code,framework_languages,content_url
0,التربية الوالدية,FMPS_C003,تعريف التربية الوالدية,FMPS_MC003_01_V01,"arabic;english,french",https://drive.google.com/file/d/19zxNklX2pIjN7...
1,التربية الوالدية,FMPS_C003,QUIZ 1.تعريف التربية الوالدية,FMPS_MC003_01_Q01,"arabic;english,french",https://docs.google.com/spreadsheets/d/1n8cpOt...
2,التربية الوالدية,FMPS_C003,أهمية وأهداف التربية الوالدية,FMPS_MC003_02_V01,"arabic;english,french",https://drive.google.com/file/d/1YptohzrIdIDrr...
3,التربية الوالدية,FMPS_C003,QUIZ 2.أهمية وأهداف التربية الوالدية,FMPS_MC003_02_Q02,"arabic;english,french",https://docs.google.com/spreadsheets/d/1-oJeml...
4,التربية الوالدية,FMPS_C003,I أساليب نجاح التربية الوالدية,FMPS_MC003_03_V01,"arabic;english,french",https://drive.google.com/file/d/1jL4QN0aU9SWtS...
5,التربية الوالدية,FMPS_C003,QUIZ 3. I أساليب نجاح التربية الوالدية,FMPS_MC003_04_Q03,"arabic;english,french",https://docs.google.com/spreadsheets/d/1sGnWUw...
6,التربية الوالدية,FMPS_C003,II أساليب نجاح التربية الوالدية,FMPS_MC003_04_V01,"arabic;english,french",https://drive.google.com/file/d/1j26FuKFZ_D4sK...
7,التربية الوالدية,FMPS_C003,QUIZ 4. II أساليب نجاح التربية الوالدية,FMPS_MC003_04_Q04,"arabic;english,french",https://docs.google.com/spreadsheets/d/15TmvNq...
8,التربية الوالدية,FMPS_C003,مشاركة الآباء وأولياء الأمور في العملية التعلي...,FMPS_MC003_05_V01,"arabic;english,french",https://drive.google.com/file/d/16-v8dAKBNWl0a...
9,التربية الوالدية,FMPS_C003,QUIZ 5.مشاركة الآباء وأولياء الأمور في العملية...,FMPS_MC003_05_Q05,"arabic;english,french",https://docs.google.com/spreadsheets/d/1ibJfUd...


In [25]:
module = module[module.columns[0:5]]
module.head()

,module_name,module_code,module_description,framework_languages,children
0,تعريف التربية الوالدية,FMPS_MC003_01,No DESCRIPTION,"arabic;english,french","QUIZ 1.تعريف التربية الوالدية,""تعريف التربية ا..."
1,أهمية وأهداف التربية الوالدية,FMPS_MC003_02,No DESCRIPTION,"arabic;english,french","QUIZ 2.أهمية وأهداف التربية الوالدية,""أهمية وأ..."
2,I أساليب نجاح التربية الوالدية,FMPS_MC003_03,No DESCRIPTION,"arabic;english,french","QUIZ 3. I أساليب نجاح التربية الوالدية,""I أسال..."
3,II أساليب نجاح التربية الوالدية,FMPS_MC003_04,No DESCRIPTION,"arabic;english,french","QUIZ 4. II أساليب نجاح التربية الوالدية,""II أس..."
4,مشاركة الآباء وأولياء الأمور في العملية التعلي...,FMPS_MC003_05,No DESCRIPTION,"arabic;english,french",QUIZ 5.مشاركة الآباء وأولياء الأمور في العملية...


In [26]:
course

,course_name,course_code,course_description,framework_languages,children
0,التربية الوالدية,FMPS_C003,التربية الوالدية مفهوم يشمل عنصرين أساسيين، يت...,lll,"تعريف التربية الوالدية,""QUIZ 1.تعريف التربية ا..."
1,نظريات التعلم,FMPS_C004,تُعد نظريات التعلم إطاراً علمياً يفسر كيفية اك...,lll,"""تعريف نظريات التعلم واهميتها"",""QUIZ 1.تعريف ن..."


In [27]:
content.head()


,course_name,course_code,content_name,content_code,framework_languages,content_url
0,التربية الوالدية,FMPS_C003,تعريف التربية الوالدية,FMPS_MC003_01_V01,"arabic;english,french",https://drive.google.com/file/d/19zxNklX2pIjN7...
1,التربية الوالدية,FMPS_C003,QUIZ 1.تعريف التربية الوالدية,FMPS_MC003_01_Q01,"arabic;english,french",https://docs.google.com/spreadsheets/d/1n8cpOt...
2,التربية الوالدية,FMPS_C003,أهمية وأهداف التربية الوالدية,FMPS_MC003_02_V01,"arabic;english,french",https://drive.google.com/file/d/1YptohzrIdIDrr...
3,التربية الوالدية,FMPS_C003,QUIZ 2.أهمية وأهداف التربية الوالدية,FMPS_MC003_02_Q02,"arabic;english,french",https://docs.google.com/spreadsheets/d/1-oJeml...
4,التربية الوالدية,FMPS_C003,I أساليب نجاح التربية الوالدية,FMPS_MC003_03_V01,"arabic;english,french",https://drive.google.com/file/d/1jL4QN0aU9SWtS...


In [28]:
content['module_code'] = content['content_code'].str.extract(r'^(FMPS_MC\d{3}_\d{2})')
content['module_code'] = content['module_code'].astype(str)
module['module_code'] = module['module_code'].astype(str)
content.shape

(36, 7)

In [29]:
merged = pd.merge(content, module, on='module_code', how='left')
merged.shape

(36, 11)

In [30]:
merged.columns

Index(['course_name', 'course_code', 'content_name', 'content_code',
       'framework_languages_x', 'content_url', 'module_code', 'module_name',
       'module_description', 'framework_languages_y', 'children'],
      dtype='object')

In [31]:
merged = merged[['course_code', 'course_name', 'module_code', 'module_name', 'content_code', 'content_name',
       'framework_languages_x', 'content_url', ]]
merged.sample(n=5)

,course_code,course_name,module_code,module_name,content_code,content_name,framework_languages_x,content_url
14,FMPS_C004,نظريات التعلم,FMPS_MC004_02,النظرية السلوكية,FMPS_MC004_02_V01,النظرية السلوكية,"arabic;english,french",https://drive.google.com/file/d/1ChYINd0_NKthF...
25,FMPS_C004,نظريات التعلم,FMPS_MC004_07,التطبيق البيداغوجي للنظرية الجشطلتية,FMPS_MC004_07_Q07,QUIZ 7. التطبيق البيداغوجي للنظرية الجشطلتية,"arabic;english,french",https://docs.google.com/spreadsheets/d/16Cgi4q...
15,FMPS_C004,نظريات التعلم,FMPS_MC004_02,النظرية السلوكية,FMPS_MC004_02_Q02,QUIZ 2.النظرية السلوكية,"arabic;english,french",https://docs.google.com/spreadsheets/d/1JIYF7h...
9,FMPS_C003,التربية الوالدية,FMPS_MC003_05,مشاركة الآباء وأولياء الأمور في العملية التعلي...,FMPS_MC003_05_Q05,QUIZ 5.مشاركة الآباء وأولياء الأمور في العملية...,"arabic;english,french",https://docs.google.com/spreadsheets/d/1ibJfUd...
28,FMPS_C004,نظريات التعلم,FMPS_MC004_09,كيفية التطبيق البيداغوجي للنظرية السوسيوبنائية,FMPS_MC004_09_V01,كيفية التطبيق البيداغوجي للنظرية السوسيوبنائية,"arabic;english,french",https://drive.google.com/file/d/1GTsONIxoJVPFJ...


In [32]:
merged = pd.merge(content_id, merged, on='content_code', how='left')

In [33]:
merged = merged[['course_code', 'course_name', 'module_code', 'module_name',
       'content_code', 'content_name', 'framework_languages_x',
    'content_url_x', 'content_do_id']]
merged.sample(n=5)

,course_code,course_name,module_code,module_name,content_code,content_name,framework_languages_x,content_url_x,content_do_id
23,FMPS_C004,نظريات التعلم,FMPS_MC004_06,النظرية الجشطالتية,FMPS_MC004_06_V01,النظرية الجشطالتية,"arabic;english,french",https://drive.google.com/file/d/1Fh-ZGbavixJH7...,do_2143097110703800321197
31,FMPS_C004,نظريات التعلم,FMPS_MC004_10,النظرية المعرفية,FMPS_MC004_10_V01,النظرية المعرفية,"arabic;english,french",https://drive.google.com/file/d/1vekWao-XjO7XI...,do_2143097136622469121201
13,FMPS_C004,نظريات التعلم,FMPS_MC004_01,تعريف نظريات التعلم واهميتها,FMPS_MC004_01_V01,تعريف نظريات التعلم واهميتها,"arabic;english,french",https://drive.google.com/file/d/1gIVQdvWiGZqMx...,do_2143097033594224641192
10,FMPS_C003,التربية الوالدية,FMPS_MC003_05,مشاركة الآباء وأولياء الأمور في العملية التعلي...,FMPS_MC003_05_V01,مشاركة الآباء وأولياء الأمور في العملية التعلي...,"arabic;english,french",https://drive.google.com/file/d/16-v8dAKBNWl0a...,do_2143096960494714881181
26,NaN,NaN,NaN,NaN,FMPS_MC004_07_Q01,NaN,NaN,https://docs.google.com/spreadsheets/d/1n8cpOt...,do_214311653141028864114


In [34]:
merged.to_json("out.json",orient="records", lines=True, force_ascii=False)

# Creating Course Hierarchy

Now let's create a hierarchical structure from the out.json file following the pattern: course -> module -> content

In [35]:
import json
import os

def create_course_hierarchy(file_path):
    # Read the JSON file line by line
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Parse each line as JSON
    json_objects = [json.loads(line) for line in lines if line.strip()]
    
    # Create the hierarchy structure
    hierarchy = {}
    
    # Process each JSON object
    for obj in json_objects:
        course_code = obj['course_code']
        course_name = obj['course_name']
        module_code = obj['module_code']
        module_name = obj['module_name']
        content_code = obj['content_code']
        
        # Initialize course if it doesn't exist
        if course_code not in hierarchy:
            hierarchy[course_code] = {
                'name': course_name,
                'modules': {}
            }
        
        # Initialize module if it doesn't exist
        if module_code not in hierarchy[course_code]['modules']:
            hierarchy[course_code]['modules'][module_code] = {
                'name': module_name,
                'contents': []
            }
        
        # Add content to the module
        hierarchy[course_code]['modules'][module_code]['contents'].append({
            'code': content_code,
            'name': obj['content_name'],
            'url': obj['content_url_x'],
            'doId': obj['content_do_id'],
            'languages': obj['framework_languages_x']
        })
    
    return hierarchy

# File path
file_path = 'out.json'

# Create hierarchy
hierarchy = create_course_hierarchy(file_path)

# Print sample to verify
course_keys = list(hierarchy.keys())
if course_keys:
    sample_course = hierarchy[course_keys[0]]
    print(f"Sample course: {sample_course['name']}")
    
    module_keys = list(sample_course['modules'].keys())
    if module_keys:
        sample_module = sample_course['modules'][module_keys[0]]
        print(f"Sample module: {sample_module['name']}")
        print(f"Number of contents: {len(sample_module['contents'])}")
        if sample_module['contents']:
            print(f"Sample content: {sample_module['contents'][0]['name']}")

Sample course: التربية الوالدية
Sample module: تعريف التربية الوالدية
Number of contents: 2
Sample content: تعريف التربية الوالدية


In [36]:
# Define descriptions for known courses
course_descriptions = {
    "FMPS_C003": "التربية الوالدية مفهوم يشمل الأسرة والمدرسة كمؤسستين اجتماعيتين تسهمان في تنشئة الطفل. تهدف إلى تحقيق التكامل بين دور الوالدين والمربي في تربية الطفل من خلال دعم القيم والسلوكيات الإيجابية، بما يعزز الجوانب النفسية والاجتماعية والمعرفية للطفل ويسهم في بناء مجتمع متماسك وواعٍ.",
    "FMPS_C004": "نظريات التعلم هي إطار نظري يفسر كيفية اكتساب المعرفة وتطوير المهارات. تهدف هذه المصوغة إلى تمكين المربي من فهم النظريات المختلفة للتعلم وتطبيقاتها في الممارسة التربوية، مما يساعده على تصميم وتنفيذ أنشطة تعليمية فعالة تناسب احتياجات المتعلمين وأنماط تعلمهم المختلفة."
}

# Function to get a default description if not found in dictionary
def get_description(course_code, course_name):
    if course_code in course_descriptions:
        return course_descriptions[course_code]
    return f"مصوغة {course_name} تهدف إلى تزويد المربي بالكفايات والمعارف الضرورية لتحسين الممارسة التربوية وتعزيز جودة التعليم الأولي."

In [37]:
# Transform hierarchy into courses.json format
courses_data = []

for course_code, course_info in hierarchy.items():
    course_name = course_info['name']
    
    # Get appropriate description for this course
    course_description = get_description(course_code, course_name)
    
    # Create a course object with the required fields
    course_obj = {
        "name": course_name,
        "description": course_description,
        "createdBy": "927c2094-987f-4e8f-8bd5-8bf93e3d2e8a",
        "organisation": "FMPS Org",
        "createdFor": ["01429195271738982411"],
        "mimeType": "application/vnd.ekstep.content-collection",
        "resourceType": "Course",
        "contentType": "Course",
        "creator": "Content Creator FMPS",
        "primaryCategory": "Course",
        "framework": "FMPS",
        "organisationIds": ["fmps_organisation_fmps"],
        "languageIds": ["fmps_language_arabic"],
        "audience": ["Student", "Teacher"],
        "targetlanguageIds": ["fmps_language_arabic", "fmps_language_french"],
        "author": "John Doe",
        "copyright": "FMPS",
        "copyrightYear": "FMPS",
        "license": "CC BY 4.0",
        "additionalCategories": ["Digital Textbook"],
        "hierarchy": {}
    }
    
    # Create modules and content mappings for the hierarchy
    for module_code, module_info in course_info['modules'].items():
        module_name = module_info['name']
        content_ids = [content['doId'] for content in module_info['contents'] if content['doId']]
        
        # Add module to hierarchy with its content IDs
        if content_ids:  # Only add modules with valid content
            course_obj["hierarchy"][module_name] = content_ids
    
    # Add course to courses array
    courses_data.append(course_obj)

# Save the transformed data to final.json
with open('final.json', 'w', encoding='utf-8') as f:
    json.dump(courses_data, f, ensure_ascii=False, indent=4)

print(f"Courses data saved to final.json")
print(f"Total courses: {len(courses_data)}")
print(f"Sample course structure:")
if courses_data:
    sample_course = courses_data[0]
    print(f"- Name: {sample_course['name']}")
    print(f"- Modules: {len(sample_course['hierarchy'])}")
    for module_name, content_ids in list(sample_course['hierarchy'].items())[:2]:
        print(f"  - {module_name}: {len(content_ids)} contents")

Courses data saved to final.json
Total courses: 3
Sample course structure:
- Name: التربية الوالدية
- Modules: 6
  - تعريف التربية الوالدية: 2 contents
  - أهمية وأهداف التربية الوالدية: 1 contents


## Verification for Node.js Scripts

The `final.json` file is now structured to match the format expected by the Node.js scripts (`1.app.js`, `2.herc.js`, and `3.revPub.js`). This format includes:

1. An array of course objects
2. Each course having the required metadata fields
3. A hierarchy property that maps module names to content IDs

This structure allows the Node.js scripts to:
- Create courses and modules via the API
- Establish hierarchical relationships
- Perform review and publishing operations

In [38]:
# Optional: Verify compatibility with courses.json format
try:
    import os
    courses_ref_path = os.path.join('..', 'courses.json')
    
    if os.path.exists(courses_ref_path):
        with open(courses_ref_path, 'r', encoding='utf-8') as f:
            reference_courses = json.load(f)
            
        print(f"Reference courses.json contains {len(reference_courses)} courses")
        print(f"Our final.json contains {len(courses_data)} courses")
        
        # Compare structure of first course
        if reference_courses and courses_data:
            ref_keys = set(reference_courses[0].keys())
            our_keys = set(courses_data[0].keys())
            
            print("\nKey comparison:")
            print(f"Keys in reference but not in ours: {ref_keys - our_keys}")
            print(f"Keys in ours but not in reference: {our_keys - ref_keys}")
            print(f"Common keys: {len(ref_keys.intersection(our_keys))}")
            
            print("\nHierarchy format:")
            print(f"Reference format: {type(reference_courses[0]['hierarchy'])}")
            print(f"Our format: {type(courses_data[0]['hierarchy'])}")
    else:
        print("Reference courses.json file not found for comparison")
except Exception as e:
    print(f"Error during verification: {e}")
    
print("\nThe final.json file is ready for use with the Node.js scripts")

Reference courses.json contains 2 courses
Our final.json contains 3 courses

Key comparison:
Keys in reference but not in ours: set()
Keys in ours but not in reference: set()
Common keys: 21

Hierarchy format:
Reference format: <class 'dict'>
Our format: <class 'dict'>

The final.json file is ready for use with the Node.js scripts


In [39]:
# Copy final.json to the parent directory for easy access by Node.js scripts
import shutil

try:
    parent_dir = os.path.join('..', 'final.json')
    shutil.copy('final.json', parent_dir)
    print(f"Copied final.json to {os.path.abspath(parent_dir)} for use by Node.js scripts")
except Exception as e:
    print(f"Error copying file: {e}")

Copied final.json to /Users/saurabhm/Downloads/ad-hoc-scripts/Protean/course_creation/final.json for use by Node.js scripts
